# Testing - Tools
## Introduction

## Installation

Note : we are pinning the inspect_ai due to a recent breaking change

In [2]:
%pip install -q openai anthropic ipywidgets colorama
import os
os.environ['XDG_RUNTIME_DIR']="/tmp"
os.environ['INSPECT_EVAL_MODEL'] = "openai/gpt-4o-mini"

from helpers.reporter.pretty import pretty_results


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Adding tools

In [3]:
from inspect_ai import Task, task, eval
from inspect_ai.dataset import Sample
from inspect_ai.solver import generate, use_tools
from inspect_ai.scorer import model_graded_fact

from inspect_ai.tool import bash, text_editor

@task
def basic_tools() -> Task:

    dataset=[
        Sample(
            input="Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.",
            target="The generated code should have the filename hello-world.js",
        )
    ]

    return Task(
        dataset=dataset,
        solver=[ 
            use_tools(bash(), text_editor()), 
            generate(tool_calls="single")
        ],
        scorer=[
            model_graded_fact()
        ],
        sandbox="docker" # indicated we run a container
    )

results = eval(basic_tools, log_level="info",display="none")
print(pretty_results(results))

Output()


Status: success Model: openai/gpt-4o-mini
/workspaces/workshop-testing/lessons/04-testing/logs/2025-05-07T13-09-15+00-00_basic-tools_PkkUnsy9QKEny46qqtgWR4.eval
======= Sample ========
input : Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.
target: The generated code should have the filename hello-world.js
 user       > Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.
 assistant [tool:text_editor] > {'command': 'create', 'path': '/repo/hello-world.js', 'file_text': "console.log('Hello, World!');"}
 assistant  > 
 tool[text_editor] > 
======== Score ======
Scorer[model_graded_fact][VALUE]: I
Scorer[model_graded_fact][EXPLANATION]: To assess the submission against the expert answer, I will follow these steps:

1. **Understanding the expert answer**: The expert states that "The generated code should have the filename hello-world.js." This indicates that the expert emphasizes the req

In [4]:
from inspect_ai import Task, task, eval
from inspect_ai.dataset import Sample
from inspect_ai.solver import generate
from inspect_ai.scorer import model_graded_fact

from inspect_ai.agent import react
from inspect_ai.tool import bash, text_editor, web_browser

@task
def react_basic_tools() -> Task:

    dataset=[
        Sample(
            input="Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.",
            target="The generated code should have the filename hello-world.js",
        )
    ]

    return Task(
        dataset=dataset,
        solver=[ 
            react(
                tools=[bash(), text_editor()]
            ),
        ],
        scorer=[
            model_graded_fact()
        ],
        sandbox="docker" # indicated we run a container
    )

results = eval(react_basic_tools, log_level="info",display="none")
print(pretty_results(results))

Output()


Status: success Model: openai/gpt-4o-mini
/workspaces/workshop-testing/lessons/04-testing/logs/2025-05-07T13-09-33+00-00_react-basic-tools_AeimHYHB3MwE7GH7MtJrc5.eval
======= Sample ========
input : Generate a javascript file name hello-world.js. Make sure to check the file was successfully created.
target: The generated code should have the filename hello-world.js
 system     > 
You are a helpful assistant attempting to submit the best possible answer.
You have several tools available to help with finding the answer. You will
see the result of tool calls right after sending the message. If you need
to perform multiple actions, you can always send more messages with additional
tool calls. Do some reasoning before your actions, describing what tool calls
you are going to use and how they fit into your plan.

When you have completed the task and have an answer, call the submit()
tool to report it.

 user       > Generate a javascript file name hello-world.js. Make sure to check the file

## Web browser

In [5]:
from inspect_ai import Task, task, eval
from inspect_ai.dataset import Sample
from inspect_ai.solver import generate
from inspect_ai.scorer import model_graded_fact

from inspect_ai.agent import react
from inspect_ai.tool import bash, text_editor, web_browser

@task
def react_basic_and_web_tools() -> Task:

    dataset=[
        Sample(
            input="Generate a javascript file name hello-world.js. Make sure to check the file was successfully created. Search the web for the best practices to create a javascript file.",
            target="The generated code should have the filename hello-world.js. ",
        )
    ]

    return Task(
        dataset=dataset,
        solver=[ 
            react(
                tools=[bash(), text_editor()]+ web_browser(interactive=False), # special case for web_browser
            ),
        ],
        scorer=[
            model_graded_fact()
        ],
        sandbox="docker" # indicated we run a container
    )

results = eval(react_basic_and_web_tools, log_level="info",display="none")
print(pretty_results(results))

Output()


Status: success Model: openai/gpt-4o-mini
/workspaces/workshop-testing/lessons/04-testing/logs/2025-05-07T13-10-07+00-00_react-basic-and-web-tools_82Ues8bqyepYvhPKFQBDKP.eval
======= Sample ========
input : Generate a javascript file name hello-world.js. Make sure to check the file was successfully created. Search the web for the best practices to create a javascript file.
target: The generated code should have the filename hello-world.js. 
 system     > 
You are a helpful assistant attempting to submit the best possible answer.
You have several tools available to help with finding the answer. You will
see the result of tool calls right after sending the message. If you need
to perform multiple actions, you can always send more messages with additional
tool calls. Do some reasoning before your actions, describing what tool calls
you are going to use and how they fit into your plan.

When you have completed the task and have an answer, call the submit()
tool to report it.

 user       